In [1]:
# Para manipulação e análise de dados
import pandas as pd
import numpy as np
import json

# Para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# analise
import spacy
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Para salvar o modelo treinado
import pickle


import json
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter


from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
dataset = 'loja_eletronico.json'
caminho_dataset = f'../datasets/{dataset}'
nlp = spacy.load("pt_core_news_md")

In [3]:
# --- Configuração ---
PERCENTUAL_TREINO = 0.8  

# --- Carregamento e Preparação ---
with open(caminho_dataset, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Criar rótulos para estratificação combinando intenção e tipos de entidade
stratify_labels = [
    f"{item['intent']}_{'_'.join(sorted(set(e['entity'] for e in item.get('entities', [])))) or 'NO_ENTITY'}"
    for item in data
]

# --- Divisão Estratificada ---
dados_treino, dados_dev = train_test_split(
    data, 
    train_size=PERCENTUAL_TREINO, 
    random_state=42, 
    stratify=stratify_labels
)

df_treino = pd.DataFrame(dados_treino)
df_dev = pd.DataFrame(dados_dev)

# --- Verificação da Distribuição dos Valores ---
def contar_valores(df):
    return Counter(v['value'] for ents in df['entities'].dropna() for v in ents)

contagem_treino = contar_valores(df_treino)
contagem_dev = contar_valores(df_dev)

df_dist_valores = pd.DataFrame({
    'Contagem Treino': pd.Series(contagem_treino),
    'Contagem Dev': pd.Series(contagem_dev)
}).fillna(0).astype(int).sort_index()

# Exibe o resultado final
print("Distribuição dos valores de entidades entre Treino e Desenvolvimento:")
display(df_dist_valores)

Distribuição dos valores de entidades entre Treino e Desenvolvimento:


,Contagem Treino,Contagem Dev
Dell XPS 15,14,2
Galaxy S25 Ultra,12,4
HP Spectre x360,14,2
MacBook Air M4,12,4
Nintendo Switch OLED,12,4
...,...,...
violões,1,0
wearables,1,0
webcams,1,0
áudio e vídeo,0,1


In [4]:
# Célula 4 - Código Completo e Corrigido

import spacy
import random
import json
from spacy.training.example import Example
from spacy.scorer import Scorer

# --- 1. CONVERSÃO DOS DADOS JÁ SEPARADOS ---

# Função auxiliar para converter o formato para o que o spaCy precisa
def converter_para_formato_spacy(dados):
    dados_formatados = []
    for item in dados:
        texto = item['text']
        entidades_validas = []
        for ent in item.get('entities', []):
            # ===================================================================
            # MUDANÇA CRUCIAL AQUI:
            # Só adiciona a entidade se ela tiver os índices 'start' e 'end'.
            # Isso garante que apenas entidades textuais sejam usadas para o treino do NER.
            # ===================================================================
            if 'start' in ent and 'end' in ent:
                entidades_validas.append((ent['start'], ent['end'], ent['entity']))
        
        # Só adiciona o exemplo se ele tiver entidades válidas para o treino
        if entidades_validas:
            dados_formatados.append((texto, {"entities": entidades_validas}))
            
    return dados_formatados

# Agora usamos as variáveis que você já criou: dados_treino e dados_dev
TRAIN_DATA = converter_para_formato_spacy(dados_treino)
DEV_DATA = converter_para_formato_spacy(dados_dev)


# --- 2. CONFIGURAÇÃO DO MODELO (Fine-Tuning) ---
DIRETORIO_SAIDA = f"../models/{dataset.replace('.json', '_finetuned_correto')}"
nlp = spacy.load("pt_core_news_md")
ner = nlp.get_pipe("ner")

# Adiciona os novos rótulos ao NER existente
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# --- 3. TREINAMENTO DO MODELO COM OS DADOS DE TREINO ---
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.select_pipes(disable=other_pipes):
    optimizer = nlp.begin_training()
    print("--- Iniciando Treinamento ---")
    # Aumentando o número de iterações para um melhor aprendizado
    for iteration in range(40): # <<-- Sugestão: Aumentar para 30 iterações
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
        # Imprime a perda a cada 5 iterações para não poluir a saída
        if (iteration + 1) % 5 == 0:
            print(f"Iteração {iteration + 1}/30 - Perda (Loss): {losses['ner']:.3f}")

# --- 4. AVALIAÇÃO COM OS DADOS DE DESENVOLVIMENTO ---
print("\n--- Avaliando o modelo com dados nunca vistos (dev set) ---")
scorer = Scorer()
examples = []
for text, annotations in DEV_DATA:
    # A predição é feita no texto puro
    doc_pred = nlp(text) 
    # O Example é criado para comparar a predição com as anotações corretas
    example = Example.from_dict(doc_pred, annotations)
    examples.append(example)

# Calcula as métricas (Precisão, Recall, F-score)
scores = scorer.score(examples)
print(f"Precisão (P): {scores['ents_p']:.2f}")
print(f"Recall (R): {scores['ents_r']:.2f}")
print(f"F-Score (F1): {scores['ents_f']:.2f}")

# --- 5. SALVAR O MODELO FINAL ---
nlp.to_disk(DIRETORIO_SAIDA)
print(f"\n✅ Modelo NER final salvo em '{DIRETORIO_SAIDA}'")

--- Iniciando Treinamento ---


c:\Users\jonat\Documents\GitHub\NLU\.env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Valor do Xbox Series X." with entities "[(9, 22, 'produto_especifico')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\jonat\Documents\GitHub\NLU\.env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Quero o Xbox Series X. A categoria é consoles de v..." with entities "[(8, 21, 'produto_especifico'), (37, 58, 'categori...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\jonat\Documents\GitHub\NLU\.env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could no

Iteração 5/30 - Perda (Loss): 36.253
Iteração 10/30 - Perda (Loss): 31.371
Iteração 15/30 - Perda (Loss): 7.366
Iteração 20/30 - Perda (Loss): 15.455
Iteração 25/30 - Perda (Loss): 4.919
Iteração 30/30 - Perda (Loss): 22.645
Iteração 35/30 - Perda (Loss): 2.000
Iteração 40/30 - Perda (Loss): 6.902

--- Avaliando o modelo com dados nunca vistos (dev set) ---


c:\Users\jonat\Documents\GitHub\NLU\.env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Na seção de consoles de videogame, estou procurand..." with entities "[(12, 33, 'categoria_produto'), (54, 67, 'produto_...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Precisão (P): 0.97
Recall (R): 0.95
F-Score (F1): 0.96

✅ Modelo NER final salvo em '../models/loja_eletronico_finetuned_correto'


In [22]:

# --- 6. TESTE INTERATIVO ---
print("\n--- Testando o modelo treinado interativamente ---")
nlp_test = spacy.load(DIRETORIO_SAIDA)
frases_teste = [
    "quanto custa um smartphone da samsung?",
    "quero comprar um notebook da apple",
    "me ve um fone de ouvido da sony",  "tchau!"]
for frase in frases_teste:
    doc = nlp_test(frase)
    print(f"\nFrase: '{frase}'")
    if doc.ents:
        for ent in doc.ents:
            print(f"  -> Entidade encontrada: '{ent.text}', Rótulo: '{ent.label_}'")
    else:
        print("  -> Nenhuma entidade encontrada.")


--- Testando o modelo treinado interativamente ---

Frase: 'quanto custa um smartphone da samsung?'
  -> Entidade encontrada: 'smartphone', Rótulo: 'categoria_produto'
  -> Entidade encontrada: 'samsung', Rótulo: 'categoria_produto'

Frase: 'quero comprar um notebook da apple'
  -> Entidade encontrada: 'notebook', Rótulo: 'categoria_produto'
  -> Entidade encontrada: 'apple', Rótulo: 'produto_especifico'

Frase: 'me ve um fone de ouvido da sony'
  -> Entidade encontrada: 'fone de ouvido', Rótulo: 'categoria_produto'
  -> Entidade encontrada: 'sony', Rótulo: 'categoria_produto'

Frase: 'tchau!'
  -> Nenhuma entidade encontrada.
